## Common Code

These steps are prerequisites to both of the models. So run these code cells before running any of the model specific code.

In [ ]:
!pip install tensorflow_io sagemaker -U

In [ ]:
import os
import sagemaker
from framework import CustomFramework

In [ ]:
role = sagemaker.get_execution_role()
print(role)

In [ ]:
inputs = {
    "train": "s3://cd2688-object-detection-tf2/train/",
    "val": "s3://cd2688-object-detection-tf2/val/"
}

In [ ]:
%%bash

git clone https://github.com/tensorflow/models.git docker/models

cp docker/models/research/object_detection/exporter_main_v2.py source_dir
cp docker/models/research/object_detection/model_main_tf2.py source_dir

In [ ]:
image_name = "tf2-object-detection-custom"
!sh ./docker/build_and_push.sh $image_name

In [ ]:
with open(os.path.join("docker", "ecr_image_fullname.txt"), "r") as f:
    container = f.readlines()[0][:-1]

print(container)

## TODO 1.1
## Model 1: Faster R-CNN ResNet152 V1 640x640

In [ ]:
tensorboard_s3_prefix = "s3://object-detection-project-singhsegv/faster-rcnn-resnet152/logs/"

In [ ]:
%%bash

mkdir /tmp/checkpoint
mkdir source_dir/checkpoint
wget -O /tmp/faster_rcnn.tar.gz http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet152_v1_640x640_coco17_tpu-8.tar.gz
tar -zxvf /tmp/faster_rcnn.tar.gz --strip-components 2 --directory source_dir/checkpoint faster_rcnn_resnet152_v1_640x640_coco17_tpu-8/checkpoint

In [ ]:
tensorboard_output_config = sagemaker.debugger.TensorBoardOutputConfig(
    s3_output_path=tensorboard_s3_prefix,
    container_local_output_path="/opt/training/"
)

estimator = CustomFramework(
    role=role,
    image_uri=container,
    entry_point="run_training.sh",
    source_dir="source_dir",
    hyperparameters={
        "model_dir": "/opt/training",
        "pipeline_config_path": "faster_rcnn_pipeline.config",
        "num_train_steps": "2000",
        "sample_1_of_n_eval_examples": "1"
    },
    instance_count=1,
    instance_type="ml.g5.xlarge",
    tensorboard_output_config=tensorboard_output_config,
    disable_profiler=True,
    base_job_name="tf2-object-detection-faster-rcnn-resnet152"
)

estimator.fit(inputs)

## TODO 1.2
## Model 2: SSD MobileNet V2 FPNLite 640x640

In [ ]:
tensorboard_s3_prefix = "s3://object-detection-project-singhsegv/ssd-mobilenet-v2-fpnlite/logs/"

In [ ]:
%%bash

mkdir /tmp/checkpoint
mkdir source_dir/checkpoint
wget -O /tmp/ssd_mobilenet_v2_fpnlite.tar.gz http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz
tar -zxvf /tmp/ssd_mobilenet_v2_fpnlite.tar.gz --strip-components 2 --directory source_dir/checkpoint ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/checkpoint

In [ ]:
tensorboard_output_config = sagemaker.debugger.TensorBoardOutputConfig(
    s3_output_path=tensorboard_s3_prefix,
    container_local_output_path="/opt/training/"
)

estimator = CustomFramework(
    role=role,
    image_uri=container,
    entry_point="run_training.sh",
    source_dir="source_dir",
    hyperparameters={
        "model_dir": "/opt/training",
        "pipeline_config_path": "ssd_mobilenet_v2_fpnlite.config",
        "num_train_steps": "2000",
        "sample_1_of_n_eval_examples": "1"
    },
    instance_count=1,
    instance_type="ml.g5.xlarge",
    tensorboard_output_config=tensorboard_output_config,
    disable_profiler=True,
    base_job_name="tf2-object-detection-ssd-mobilenet-v2-fpnlite"
)

estimator.fit(inputs)